In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Load all markdown files from chapters directory
loader = DirectoryLoader('./chapters', glob="**/*.md")
documents = loader.load()

# Add counter index to each document
for i, doc in enumerate(documents):
    doc.metadata['index'] = i

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
texts = text_splitter.split_documents(documents)

# Preserve the original document index in the chunks
for text in texts:
    text.metadata['doc_index'] = text.metadata['index']
# Add chunk index and master index to each text
master_index = 0
for doc_index in range(max(text.metadata['doc_index'] for text in texts) + 1):
    # Get all chunks for this document
    doc_chunks = [t for t in texts if t.metadata['doc_index'] == doc_index]
    
    # Add chunk index within document
    for chunk_idx, chunk in enumerate(doc_chunks):
        chunk.metadata['chunk_index'] = chunk_idx
        chunk.metadata['master_index'] = master_index
        master_index += 1

# Create embeddings and store in Chroma vector database
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

# Create retriever interface
retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
)

In [ ]:
# Create chat interface with memory
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create system prompt for story assistant
system_prompt = """You are a knowledgeable assistant who helps answer questions about the story's history and lore. 
Use the provided context to give accurate and detailed responses about the story's world, characters, and events.
If you're not sure about something, be honest about your uncertainty."""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

while True:
    # Get user input
    user_input = input("\nAsk me about the story (or type 'quit' to exit): ")
    if user_input.lower() == 'quit':
        break
        
    # Search relevant context
    context_docs = retriever.get_relevant_documents(user_input)
    context = "\n\n".join([doc.page_content for doc in context_docs])
    
    # Create messages with context
    response = prompt.format(
        chat_history=memory.chat_memory.messages,
        input=f"Using this context:\n{context}\n\nAnswer this question: {user_input}"
    )
    
    print("\nAssistant:", response)
    
    # Store conversation in memory
    memory.chat_memory.add_user_message(user_input)
    memory.chat_memory.add_ai_message(response)

In [ ]:
# Create chains for memory processing and story generation
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import io

# Initialize LLMs with different temperatures for different creative needs
experience_llm = ChatOpenAI(temperature=0.7)
story_llm = ChatOpenAI(temperature=0.9)
latex_llm = ChatOpenAI(temperature=0.3)

# Create prompts for each stage
experience_prompt = PromptTemplate(
    input_variables=["context"],
    template="""Given these story elements and memories:

{context}

Suggest deeply human experiences, emotions, and universal themes that relate to these elements. 
Consider fundamental human experiences like love, loss, growth, fear, triumph, etc.
Be specific and draw meaningful connections."""
)

story_prompt = PromptTemplate(
    input_variables=["experiences", "related_memories"],
    template="""Using these human experiences and thematic elements:

{experiences}

And drawing from these related memories and story elements:

{related_memories}

Craft a rich and engaging story segment that weaves these elements together.
Focus on vivid imagery, emotional resonance, and narrative flow."""
)

latex_prompt = PromptTemplate(
    input_variables=["story"],
    template="""Convert this story segment into properly formatted LaTeX:

{story}

Use appropriate LaTeX formatting for literary text, including proper paragraph breaks,
quotation marks, and any needed structural elements."""
)

# Create the processing chains
experience_chain = LLMChain(llm=experience_llm, prompt=experience_prompt)
story_chain = LLMChain(llm=story_llm, prompt=story_prompt)
latex_chain = LLMChain(llm=latex_llm, prompt=latex_prompt)

# Create output stream
output_file = io.StringIO()

# Get all documents and their indices
all_docs = retriever.get_relevant_documents("")
doc_indices = {doc.page_content: idx for idx, doc in enumerate(all_docs)}

# Process each document chunk
for current_idx, current_doc in enumerate(all_docs):
    current_content = current_doc.page_content
    
    # Only retrieve documents that came before this one
    related_docs = [doc for doc in all_docs 
                   if doc_indices[doc.page_content] < current_idx]
    related_memories = "\n\n".join([doc.page_content for doc in related_docs])
    
    # Combine current content with related memories for context
    full_context = f"Current Memory:\n{current_content}\n\nRelated Memories:\n{related_memories}"
    
    # Generate human experiences
    experiences = experience_chain.run(context=full_context)
    
    # Generate story
    story = story_chain.run(experiences=experiences, related_memories=related_memories)
    
    # Convert to LaTeX
    latex_text = latex_chain.run(story=story)
    
    # Write to output stream
    output_file.write(latex_text + "\n\n")

# Get final output
final_latex = output_file.getvalue()
print("Generated LaTeX Story:")
print(final_latex)